In [1]:
!pip install -q git+https://github.com/tensorflow/docs

  Preparing metadata (setup.py) ... done


In [2]:
import os
import keras
from keras import layers
from keras.applications.densenet import DenseNet121
from tensorflow_docs.vis import embed
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import imageio
import cv2

In [11]:
MAX_SEQ_LENGTH = 20
NUM_FEATURES = 1024
IMG_SIZE = 128

EPOCHS = 200

In [4]:
import os
import numpy as np
import cv2
from tensorflow import keras
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import CenterCrop, Input
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split


IMG_SIZE = 224
MAX_SEQ_LENGTH = 120
NUM_FEATURES = 1024


center_crop_layer = keras.layers.CenterCrop(IMG_SIZE, IMG_SIZE)

def crop_center(frame):
    cropped = center_crop_layer(frame[None, ...])
    cropped = keras.backend.squeeze(cropped, axis=0)
    return cropped.numpy()

def load_video(path, max_frames=0, offload_to_cpu=False):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = frame[:, :, [2, 1, 0]]  # BGR到RGB
            frame = crop_center(frame)
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

def build_feature_extractor():
    feature_extractor = DenseNet121(weights="imagenet", include_top=False, pooling="avg", input_shape=(IMG_SIZE, IMG_SIZE, 3))
    preprocess_input = keras.applications.densenet.preprocess_input
    inputs = Input((IMG_SIZE, IMG_SIZE, 3))
    x = preprocess_input(inputs)
    outputs = feature_extractor(x)
    model = Model(inputs, outputs, name="feature_extractor")
    return model

feature_extractor = build_feature_extractor()

def prepare_videos(root_dir):
    frame_features = []
    labels = []
    for label in ['0', '1']:
        class_dir = os.path.join(root_dir, label)
        for video_file in os.listdir(class_dir):
            video_path = os.path.join(class_dir, video_file)
            frames = load_video(video_path, max_frames=MAX_SEQ_LENGTH)
            frames_feature = np.zeros((MAX_SEQ_LENGTH, NUM_FEATURES))
            for i, frame in enumerate(frames):
                if i < MAX_SEQ_LENGTH:
                    frame = np.expand_dims(frame, axis=0)  # 添加批次维度
                    frames_feature[i] = feature_extractor.predict(frame)
            frame_features.append(frames_feature)
            labels.append(int(label))
    frame_features = np.array(frame_features)
    labels = np.array(labels)
    return frame_features, labels

root_dir = '/content/drive/MyDrive/ViolenceDataPaper2'

frame_features, labels = prepare_videos(root_dir)


X_train, X_test, y_train, y_test = train_test_split(frame_features, labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)  # 0.25 * 0.8 = 0.2

print(f"训练集特征维度: {X_train.shape}, 训练集标签维度: {y_train.shape}")
print(f"验证集特征维度: {X_val.shape}, 验证集标签维度: {y_val.shape}")
print(f"测试集特征维度: {X_test.shape}, 测试集标签维度: {y_test.shape}")


串流輸出內容已截斷至最後 5000 行。
1/1 [==============================] - 0s 28ms/step
训练集特征维度: (147, 120, 1024), 训练集标签维度: (147,)
验证集特征维度: (50, 120, 1024), 验证集标签维度: (50,)
测试集特征维度: (50, 120, 1024), 测试集标签维度: (50,)


In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import Adam

In [7]:
class PositionalEmbedding(nn.Module):
    def __init__(self, sequence_length, embed_size):
        super(PositionalEmbedding, self).__init__()
        self.position_embeddings = nn.Embedding(num_embeddings=sequence_length, embedding_dim=embed_size)

    def forward(self, x):
        positions = torch.arange(0, x.size(1)).unsqueeze(0).to(x.device)
        x = x + self.position_embeddings(positions)
        return x


In [8]:
class TransformerEncoder(nn.Module):
    def __init__(self, embed_size, dense_dim, num_heads):
        super(TransformerEncoder, self).__init__()
        self.attention = nn.MultiheadAttention(embed_dim=embed_size, num_heads=num_heads, dropout=0.3)
        self.dense_proj = nn.Sequential(
            nn.Linear(embed_size, dense_dim),
            nn.GELU(),
            nn.Linear(dense_dim, embed_size)
        )
        self.layernorm1 = nn.LayerNorm(embed_size)
        self.layernorm2 = nn.LayerNorm(embed_size)

    def forward(self, src):
        src_attended, _ = self.attention(src, src, src)
        src = self.layernorm1(src + src_attended)
        src_proj = self.dense_proj(src)
        src = self.layernorm2(src + src_proj)
        return src


In [9]:
class VideoClassifier(nn.Module):
    def __init__(self, sequence_length, num_features, embed_size, dense_dim, num_heads, num_classes):
        super(VideoClassifier, self).__init__()
        self.pos_embedding = PositionalEmbedding(sequence_length, num_features)
        self.transformer_encoder = TransformerEncoder(embed_size, dense_dim, num_heads)
        self.global_pool = nn.AdaptiveAvgPool1d(1)
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(num_features, num_classes)

    def forward(self, x):
        x = self.pos_embedding(x)
        x = x.permute(1, 0, 2)  # Transformer expects (Seq Len, Batch, Features)
        x = self.transformer_encoder(x)
        x = x.permute(1, 2, 0)  # Pooling expects (Batch, Features, Seq Len)
        x = self.global_pool(x).squeeze(2)
        x = self.dropout(x)
        x = self.fc(x)
        return x


In [9]:
EPOCHS = 5

In [10]:

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor.view(-1, 1))
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)

val_dataset = TensorDataset(X_val_tensor, y_val_tensor.view(-1, 1))
val_loader = DataLoader(dataset=val_dataset, batch_size=64, shuffle=False)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor.view(-1, 1))
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)


In [11]:
def train_model(model, train_loader, val_loader, num_epochs):
    optimizer = Adam(model.parameters(), lr=1e-3)
    criterion = nn.BCEWithLogitsLoss()

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # 验证逻辑
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for inputs, targets in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)
        avg_val_loss = val_loss / len(val_loader)
        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}')

    return model



In [12]:

model = VideoClassifier(
    sequence_length=MAX_SEQ_LENGTH,
    num_features=NUM_FEATURES,
    embed_size=1024,
    dense_dim=128,
    num_heads=4,
    num_classes=1
)


In [13]:
# 确保 EPOCHS 变量已经被设置
EPOCHS = 240  # 或根据您的需求调整

# 开始训练模型
trained_model = train_model(model, train_loader, val_loader, EPOCHS)


Epoch 1/240, Train Loss: 0.8717, Val Loss: 0.5720
Epoch 2/240, Train Loss: 0.6470, Val Loss: 0.7064
Epoch 3/240, Train Loss: 0.6640, Val Loss: 0.5373
Epoch 4/240, Train Loss: 0.4191, Val Loss: 0.3196
Epoch 5/240, Train Loss: 0.2714, Val Loss: 0.2321
Epoch 6/240, Train Loss: 0.2819, Val Loss: 0.2711
Epoch 7/240, Train Loss: 0.1873, Val Loss: 1.2038
Epoch 8/240, Train Loss: 0.2727, Val Loss: 0.3395
Epoch 9/240, Train Loss: 0.1821, Val Loss: 0.5460
Epoch 10/240, Train Loss: 0.1934, Val Loss: 0.1172
Epoch 11/240, Train Loss: 0.1309, Val Loss: 0.2090
Epoch 12/240, Train Loss: 0.1381, Val Loss: 0.0419
Epoch 13/240, Train Loss: 0.1161, Val Loss: 0.1506
Epoch 14/240, Train Loss: 0.0933, Val Loss: 0.1201
Epoch 15/240, Train Loss: 0.0499, Val Loss: 0.0268
Epoch 16/240, Train Loss: 0.0363, Val Loss: 0.1716
Epoch 17/240, Train Loss: 0.0567, Val Loss: 0.0581
Epoch 18/240, Train Loss: 0.0424, Val Loss: 0.0935
Epoch 19/240, Train Loss: 0.0154, Val Loss: 0.1018
Epoch 20/240, Train Loss: 0.0113, Val Lo

In [14]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
def evaluate_model(model, test_loader):
    model.eval()
    y_true = []
    y_pred = []

    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)
            predictions = torch.round(torch.sigmoid(outputs))
            y_true.extend(targets.cpu().numpy())
            y_pred.extend(predictions.cpu().numpy())

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1 Score: {f1:.4f}')


In [15]:
evaluate_model(trained_model, test_loader)


Accuracy: 0.9400
Precision: 0.9200
Recall: 0.9583
F1 Score: 0.9388


In [16]:
# 保存模型
torch.save(trained_model, 'model_complete.pth')



In [17]:
import torch


In [18]:
model = torch.load('/content/model_complete.pth')
model.eval()  # 切换到评估模式

VideoClassifier(
  (pos_embedding): PositionalEmbedding(
    (position_embeddings): Embedding(120, 1024)
  )
  (transformer_encoder): TransformerEncoder(
    (attention): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
    )
    (dense_proj): Sequential(
      (0): Linear(in_features=1024, out_features=128, bias=True)
      (1): GELU(approximate='none')
      (2): Linear(in_features=128, out_features=1024, bias=True)
    )
    (layernorm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (layernorm2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (global_pool): AdaptiveAvgPool1d(output_size=1)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=1024, out_features=1, bias=True)
)

In [2]:
import torch

In [13]:
from google.colab import drive
drive.mount('/content/drive')

model_path = '/content/drive/My Drive/model_complete.pth'

# 假设 'model' 是您要保存的 PyTorch 模型
import torch
torch.save(model.state_dict(), model_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
